In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('inputMelodiesAugmented.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("unique vocabulary")
print(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
#print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

unique vocabulary
['\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'R', 'a', 'c', 'd', 'f', 'g']
10.74971 M parameters
step 0: train loss 2.6962, val loss 2.6865
step 500: train loss 1.6796, val loss 1.6102
step 1000: train loss 1.3363, val loss 1.3167
step 1500: train loss 1.1942, val loss 1.2047
step 2000: train loss 1.1087, val loss 1.1500
step 2500: train loss 1.0230, val loss 1.1508
step 3000: train loss 0.9260, val loss 1.1711
step 3500: train loss 0.8073, val loss 1.2351
step 4000: train loss 0.6774, val loss 1.3119
step 4500: train loss 0.5621, val loss 1.4189
step 4999: train loss 0.4624, val loss 1.5578


In [ ]:
model = GPTLanguageModel()
model.load_state_dict(torch.load('gpt_melody_model.pth'))
model.to(device)


<ipython-input-4-c77817e52081>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('gpt_melody_model.pth'))


GPTLanguageModel(
  (token_embedding_table): Embedding(14, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=

In [ ]:
with open('inputMelodiesAugmented.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
print("unique vocabulary")
print(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


unique vocabulary
['\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'R', 'a', 'c', 'd', 'f', 'g']


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class MelodyDataset(Dataset):
    def __init__(self, data, block_size):
        """
        Args:
            data (torch.Tensor): Encoded dataset as a 1D tensor.
            block_size (int): The context length for GPT.
        """
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        """
        Returns:
            x (torch.Tensor): Input sequence of length block_size.
            y (torch.Tensor): Target sequence (next token for each input token).
        """
        x = torch.tensor(self.data[idx:idx + self.block_size], dtype=torch.long)
        y = torch.tensor(self.data[idx + 1:idx + self.block_size + 1], dtype=torch.long)
        return x, y


In [ ]:

block_size = 256
batch_size = 64   # Number of sequences in a batch
dataset = MelodyDataset(encode(text), block_size)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [1]:

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_melody = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print("Generated Melody:", generated_melody)


Generated Melody: FCaCCdCFgFdFgFRFFCaCCFddCaCagCCdCagFFRFCaCCdCdCagFFRFFggFgFaCCdCdCaggFFRFFggFgFaCCdCdCagFFRFFggFgFaCCdCdCagFFRFCaCCFdRCCCdCagCCdCagFFRFFggFgFgFaCCdCdCagFFRFFFggFgFgFaCCdCdCagFFRFFggFgFaCCdCdCagRcCaa


In [ ]:
strings = "FCaCCdCFgFdFgFRFFCaCCFddCaCagCCdCagFFRFCaCCdCdCagFFRFFggFgFaCCdCdCaggFFRFFggFgFaCCdCdCagFFRFFggFgFaCCdCdCagFFRFCaCCFdRCCCdCagCCdCagFFRFFggFgFgFaCCdCdCagFFRFFFggFgFgFaCCdCdCagFFRFFggFgFaCCdCdCagRcCaa"


In [ ]:
def calculate_perplexity(model, data_loader, device):
    model.eval()
    total_loss = 0
    count = 0
    criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)  # Move tensors to the device
            logits = model(x)

            # If logits is a tuple, get the first element
            if isinstance(logits, tuple):
                logits = logits[0]

            # Compute the loss
            loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            total_loss += loss.item() * x.size(0)
            count += x.size(0)

    avg_loss = total_loss / count
    perplexity = torch.exp(torch.tensor(avg_loss))
    return perplexity.item()
perplexity = calculate_perplexity(model, data_loader, device)
print(f"Pitch-only Perplexity: {perplexity}")

Pitch-only Perplexity: 1.770843505859375


In [ ]:
#for pitch diversity u can create a table with three columns, one for original and one for other two settingsof hyperparamters
def calculate_pitch_diversity(generated_melody):
    unique_pitches = set(generated_melody)
    print((unique_pitches))
    diversity_score = len(unique_pitches) / len(generated_melody)
    return diversity_score

pitch_diversity = calculate_pitch_diversity("FCaCCdCFgFdFgFRFFCaCCFddCaCagCCdCagFFRFCaCCdCdCagFFRFFggFgFaCCdCdCaggFFRFFggFgFaCCdCdCagFFRFFggFgFaCCdCdCagFFRFCaCCFdRCCCdCagCCdCagFFRFFggFgFgFaCCdCdCagFFRFFFggFgFgFaCCdCdCagFFRFFggFgFaCCdCdCagRcCaa")
print(f"Pitch Diversity: {pitch_diversity}")


{'a', 'C', 'c', 'F', 'd', 'g', 'R'}
Pitch Diversity: 0.03535353535353535


In [ ]:
import torch
from collections import defaultdict
import math
with open('inputMelodiesAugmented.txt', 'r', encoding='utf-8') as f:
    text = f.read()
class BigramModel:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.bigram_counts = defaultdict(lambda: defaultdict(int))
        self.total_counts = defaultdict(int)

    def train(self, sequences):
        # Count bigram occurrences
        for sequence in sequences:
            for i in range(len(sequence) - 1):
                self.bigram_counts[sequence[i]][sequence[i + 1]] += 1
                self.total_counts[sequence[i]] += 1

    def predict_next(self, prev_token):
        # Predict next token probabilities
        token_counts = self.bigram_counts[prev_token]
        total = self.total_counts[prev_token]
        if total == 0:
            return torch.ones(self.vocab_size) / self.vocab_size  # Uniform distribution
        probabilities = torch.zeros(self.vocab_size)
        for token, count in token_counts.items():
            probabilities[token] = count / total
        return probabilities

    def perplexity(self, data):
        log_prob = 0
        total_tokens = 0
        for sequence in data:
            for i in range(len(sequence) - 1):
                prev_token = sequence[i]
                next_token = sequence[i + 1]
                probabilities = self.predict_next(prev_token)
                log_prob += math.log(probabilities[next_token].item() + 1e-9)
                total_tokens += 1
        avg_log_prob = log_prob / total_tokens
        return math.exp(-avg_log_prob)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("unique vocabulary")
print(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Preprocess pitch-only data for Bigram Model
encoded_data = torch.tensor(encode(text), dtype=torch.long)
train_sequences = [encoded_data[i:i + 256] for i in range(0, len(encoded_data) - 256, 256)]

# Train Bigram Model
bigram_model = BigramModel(vocab_size)
bigram_model.train(train_sequences)

# Evaluate Bigram Model Perplexity
train_perplexity = bigram_model.perplexity(train_sequences)
print(f"Bigram Model Train Perplexity: {train_perplexity:.4f}")


unique vocabulary
['\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'R', 'a', 'c', 'd', 'f', 'g']
Bigram Model Train Perplexity: 14.0000


In [ ]:
import torch.nn.functional as F

n = int(0.9 * len(encoded_data))
train_data = encoded_data[:n]
val_data = encoded_data[n:]

# Print the lengths of the splits
print(f"Train Data Length: {len(train_data)}")
print(f"Validation Data Length: {len(val_data)}")

def calculate_bigram_loss(model, sequences):
    total_loss = 0
    total_tokens = 0

    for sequence in sequences:
        for i in range(len(sequence) - 1):
            prev_token = sequence[i]
            next_token = sequence[i + 1]
            predicted_probs = model.predict_next(prev_token)
            loss = -torch.log(predicted_probs[next_token] + 1e-9)
            total_loss += loss.item()
            total_tokens += 1

    avg_loss = total_loss / total_tokens
    return avg_loss

train_loss_bigram = calculate_bigram_loss(bigram_model, [train_data.tolist()])
val_loss_bigram = calculate_bigram_loss(bigram_model, [val_data.tolist()])

print(f"Bigram Model Train Loss: {train_loss_bigram:.4f}")
print(f"Bigram Model Validation Loss: {val_loss_bigram:.4f}")


Train Data Length: 639139
Validation Data Length: 71016
Bigram Model Train Loss: 2.6391
Bigram Model Validation Loss: 2.6391


In [ ]:
def generate_bigram_sequence(model, length, start_token):
    sequence = [start_token]
    for _ in range(length - 1):
        prev_token = sequence[-1]
        next_token_probs = model.predict_next(prev_token)
        next_token = torch.multinomial(next_token_probs, num_samples=1).item()
        sequence.append(next_token)
    return sequence

start_token = stoi['C']  # Start with 'C' (or any token in the dataset)
generated_sequence = generate_bigram_sequence(bigram_model, length=256, start_token=start_token)
generated_sequence_str = ''.join([itos[token] for token in generated_sequence])
print("Generated Sequence:", generated_sequence_str)


Generated Sequence: CAfBBEfRagdAGDFaEagCaFDCGR
EAEfECcFEACcfaAfBCFBAGafffBaRAFCgAa
DEF
acCDCaCBfFfA
a
EEAE
DR
dCCCfffcFcdEDaff
BDBGDGDB
BCDG
A
ACcEERARBCcd
DBfFBcFdDdDaCa
EfEfagfBRDaCDgcAcfcERfaADdECCGcdfG
gDdFEgBf
CdfDREEffRBccafgBGfdgfgGB
fFFAGfaBgdfBgADaCBdEadFC
BdEccadcGa


In [ ]:
calculate_pitch_diversity("EfEfagfBRDaCDgcAcfcERfaADdECCGcdfG")

{'a', 'D', 'f', 'C', 'c', 'E', 'B', 'A', 'd', 'g', 'G', 'R'}


0.35294117647058826

In [ ]:
calculate_pitch_diversity("RAgBAAgBAfDBARDccBBAAfGAAERBacBGfAGEdfERABcDEfGBAGfRAgBAAgBAfDBARDccBBAAfGABRGAaaCDCaaARBAAcRBAAfRGABBcDcBBAABAGAfEGAfEEDRAgBAAgBAfDBARDccBBAAfGABRGAaaCDCaaARBAAcRBAAfRGABBcDcBBAABAGAfEGAfEEDRGAaaAcED")

{'a', 'D', 'f', 'c', 'C', 'E', 'B', 'A', 'd', 'g', 'G', 'R'}


0.06